In [1]:
import os
from collections import defaultdict
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

In [ ]:
class CNN_BERT(nn.Module):

    def __init__(self):
        super(CNN_BERT, self).__init__()


    def forward(self,x):
        return x


In [ ]:
# class CNN_GC(nn.Module):

#     def __init__(self):
#         super(CNN_GC, self).__init__()
        

#     def forward(self,x):
#         return x


In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.l0 = nn.Linear(300,300)
        self.dp0 = nn.Dropout(p=0.5)
        self.l1 = nn.Linear(300,300)
        self.dp1 = nn.Dropout(p=0.5)
        self.l3 = nn.Linear(300,2)

    def forward(self,x):
        x = self.dp0(F.sigmoid(self.l0(x)))
        x = self.dp1(F.sigmoid(self.l1(x)))
        x = F.sigmoid(self.l3(x))
        return x

In [3]:
class SuggestionClassifier(nn.Module):
    def __init_(self):
        # path all the models together
        # BERT->CNN + CNN->ATT
        super(SuggestionClassifier,self).__init__()
        self.CNN_b = CNN_BERT()
        self.CNN_gc = CNN_GC()
        self.MLP = MLP()


    def forward(self,x):
        bert_x = self.CNN_b(x)
        gc_x = self.CNN_gc(x)
        torch.cat(bert_x,gc_x) # TODO : ensure dimensions align
        x = self.MLP(x)
        return x